# How to Use Deep Reinforcement Learning to Improve your Supply Chain

Full write up available [here](https://www.datahubbs.com/how-to-use-deep-reinforcement-learning-to-improve-your-supply-chain/).

In [4]:
import or_gym
from or_gym.utils import create_env
import ray
from ray.rllib.agents.ppo import PPOTrainer
from ray import tune
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [19]:
def register_env(env_name, env_config={}):
    env = create_env(env_name)
    tune.register_env(env_name, 
        lambda env_name: env(env_name,
            env_config=env_config))

# Environment and RL Configuration Settings
env_name = 'InvManagement-v1'
# env_name = "Knapsack-v0"
env_config = {} # Change environment parameters here
rl_config = dict(
    env=env_name,
    num_workers=2,
    env_config=env_config,
    model=dict(
        vf_share_layers=False,
        fcnet_activation='elu',
        fcnet_hiddens=[256, 256]
    ),
    lr=1e-5
)
 
# Register environment
register_env(env_name, env_config)

In [20]:
# Initialize Ray and Build Agent
ray.init(ignore_reinit_error=True)
agent = PPOTrainer(env=env_name,
    config=rl_config)
 
results = []
for i in range(500):
    res = agent.train()
    results.append(res)
    if (i+1) % 5 == 0:
        print('\rIter: {}\tReward: {:.2f}'.format(
                i+1, res['episode_reward_mean']), end='')
ray.shutdown()

(RolloutWorker pid=32512) 2022-08-17 16:32:57,950	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=32513) 2022-08-17 16:32:57,936	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
(RolloutWorker pid=32512) 2022-08-17 16:32:58,260	ERROR worker.py:451 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=32512, ip=192.168.1.177, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f220d2cf9a0>)
(RolloutWorker pid=32512)   File "/home/christian/a

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=32512, ip=192.168.1.177, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x7f220d2cf9a0>)
  File "/home/christian/anaconda3/envs/or-gym/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 630, in __init__
    self._build_policy_map(
  File "/home/christian/anaconda3/envs/or-gym/lib/python3.9/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1788, in _build_policy_map
    self.policy_map.create_policy(
  File "/home/christian/anaconda3/envs/or-gym/lib/python3.9/site-packages/ray/rllib/policy/policy_map.py", line 140, in create_policy
    self[policy_id] = class_(
  File "/home/christian/anaconda3/envs/or-gym/lib/python3.9/site-packages/ray/rllib/policy/tf_policy_template.py", line 256, in __init__
    DynamicTFPolicy.__init__(
  File "/home/christian/anaconda3/envs/or-gym/lib/python3.9/site-packages/ray/rllib/policy

In [16]:
env = or_gym.make("InvManagement-v0")
env.action_space

Box([0 0 0], [100  90  80], (3,), int16)

In [25]:
low_ = np.min(env.action_space.low)
high_ = np.max(env.action_space.high)

assert np.all(env.action_space.low == low_)
assert np.all(env.action_space.high == high_)

AssertionError: 

In [27]:
high_, env.action_space.high

(100, array([100,  90,  80], dtype=int16))

In [17]:
    def required_model_output_shape(
        action_space: gym.Space, model_config: ModelConfigDict
    ) -> Union[int, np.ndarray]:
        # Int Box.
        if isinstance(action_space, gym.spaces.Box):
            assert action_space.dtype.name.startswith("int")
            low_ = np.min(action_space.low)
            high_ = np.max(action_space.high)
            assert np.all(action_space.low == low_)
            assert np.all(action_space.high == high_)
            np.prod(action_space.shape, dtype=np.int32) * (high_ - low_ + 1)
        # MultiDiscrete space.
        else:
            # nvec is already integer, so no casting needed.
            return np.sum(action_space.nvec)

array([100,  90,  80], dtype=int16)

In [30]:
env.action_space.nvec

AttributeError: 'Box' object has no attribute 'nvec'

: 

In [29]:
low_

0